In [1]:
import pandas as pd 
from difflib import SequenceMatcher as SM
df_club=pd.read_csv("Datasets\Clubs_data.csv")
df_meta=pd.read_csv("Datasets\Metadata.csv")
df_organiser=pd.read_csv("Datasets\Organisers_In_Fests.csv")
df_participants=pd.read_csv("Datasets\Participants_In_Fests.csv")

In [2]:
def is_similarity(first, second, ratio):
    return SM(None, first, second).ratio() > ratio

In [3]:
#For Club name correction
for Index, row in df_club.iterrows():
    # print(Index, len(df_club.iloc[Index]["Name"]))
    df_club.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.64):
            df_club.at[Index, "Name"] = row2["Name"]
            

In [26]:
#Removing Invalid Names in Clubs CSV
list_small_5 = []
for Index, row in df_club.iterrows():
    if len(df_club.iloc[Index]["Name"]) < 7:
        list_small_5.append(Index)
df_club = df_club.drop(df_club.index[list_small_5])

In [4]:
#For Participants Name correction
for Index, row in df_participants.iterrows():
    df_participants.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.59):
            df_participants.at[Index, "Name"] = row2["Name"]

In [5]:
#Removing Invalid Names in Participations CSV
list_small_7 = []
for Index, row in df_participants.iterrows():
    if len(df_participants.iloc[Index]["Name"]) < 7:
        list_small_7.append(Index)
df_participants = df_participants.drop(df_participants.index[list_small_7])

In [6]:
#For Organisers Name Correction
for Index, row in df_organiser.iterrows():
    df_organiser.at[Index, "Name"] = row["Name"].lower()
    for Index2, row2 in df_meta.iterrows():
        df_meta.at[Index2, "Name"] = row2["Name"].lower()
        if is_similarity(row["Name"], row2["Name"], 0.69):
            df_organiser.at[Index, "Name"] = row2["Name"]

In [9]:
#Removing Invalid Names in Organisers CSV
list_small_7 = []
for Index, row in df_organiser.iterrows():
    if len(df_organiser.iloc[Index]["Name"]) < 7:
        list_small_7.append(Index)
df_organiser = df_organiser.drop(df_organiser.index[list_small_7])

In [11]:
#Importing Numpy and creating a Combined DF with all the data
import numpy as np
df_combined = pd.concat([df_club,df_organiser,df_participants], ignore_index=True)
df_combined['Roll No.'] = np.nan

In [14]:
#Assigning Roll No. to people for making access easier
for Index, row in df_combined.iterrows():
    for Index2, row2 in df_meta.iterrows():
        if row["Name"] == row2["Name"]:
            df_combined.at[Index, "Roll No."] = row2["ID"]
df_combined = df_combined.sort_values(by="Roll No.")
df_combined_No_Null = df_combined[df_combined["Roll No."].notnull()]

In [15]:
new_cols = ["Roll_No", "Name", "Club_Name", "Fest_Name" ,"Event", "Role"]
df_combined_No_Null=df_combined_No_Null.reindex(columns=new_cols)

In [16]:
df_combined_No_Null.head()

,Roll No.,Name,Club_Name,Fest_Name,Event,Role
1773,17XJ1A0100,monet wallace,NaN,fest_2,fest_2_event_5,NaN
1464,17XJ1A0100,monet wallace,NaN,fest_2,fest_2_event_3,NaN
995,17XJ1A0100,monet wallace,club_2,NaN,club_2_event_1,Participant
2924,17XJ1A0100,monet wallace,NaN,fest_1,fest_1_event_1,NaN
5020,17XJ1A0100,monet wallace,NaN,fest_1,fest_1_event_4,NaN


In [ ]:
[
    {
        'name': 'Dummy',
        'id': 'id000',
        'clubs': {
            'club_i': {
                'isOrganiser': 'Organiser',
                'club_i_event_j': {
                    'participated': False
                    },
            },
        },
        'fests': {
            'fest_i': {
                'isOrganiser': '',
                'fest_i_event_j': {
                    'participated': True
                    },
            },
        }
    }
]

In [ ]:
def pd_to_clubs_dict(df):
    for (roll_No), df_cus_grouped in df.groupby(["Roll_No"])